## 1. **Task: Bank Transactions**
You are tasked with implementing a banking system that allows customers to transfer funds between accounts.
Design a SQL transaction to ensure that the amount is deducted from the sender's account and added to the receiver's account in an atomic manner.
If any part of the transaction fails, rollback the changes to maintain data integrity.

### Import library

In [1]:
import mysql.connector

### Establishing connection to MySQL

In [2]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='bank')

### Creating a cursor object

In [3]:
cursor = connection.cursor()

### Creating tables

In [4]:
customers_table = ("""
CREATE TABLE customers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    surname VARCHAR(50),
    balance INT
    );
""")

transactions_table = ("""
CREATE TABLE transactions (
    id INT AUTO_INCREMENT PRIMARY KEY,
    sender INT,
    receiver INT,
    balance INT
    );
""")

cursor.execute(customers_table)
cursor.execute(transactions_table)

### Inserting values into tables

In [5]:
cursor.execute("""
INSERT INTO customers (
    name, surname, balance
    ) VALUES                  
        ("Johny", "Bravo", 500),
        ("Michael", "Keaton", 987892),
        ("Philip", "Smith", 999),
        ("Jack", "Bing",456321),
        ("Velma", "Vellington", 1000000);
""")

In [6]:
connection.commit()

### Transaction:

In [7]:
def transaction (sender_id:int,receiver_id:int,amount:int):
    try:
        cursor.execute("SELECT balance FROM customers WHERE id = %s;",(sender_id,))
        sender_balance = cursor.fetchone()[0]
        
        print(f"Sender balance:",sender_balance)
        print(f"Amount to be transfered:",amount)
        
        
        if amount <= sender_balance:
            
            # Updating sender balance
            sender_balance -= amount
            cursor.execute("UPDATE customers SET balance = %s WHERE id = %s;",(sender_balance, sender_id,))
            
            # Updating receiver balance
            cursor.execute("SELECT balance FROM customers WHERE id = %s;",(receiver_id,))
            receiver_balance = cursor.fetchone()[0]
            print(f"Receiver balance before transaction:",receiver_balance)
            
            receiver_balance += amount
            cursor.execute("UPDATE customers SET balance = %s WHERE id = %s;",(receiver_balance, receiver_id,))  
            
            # Updating transactions table
            cursor.execute("INSERT INTO transactions (sender, receiver, balance) VALUES (%s,%s,%s)",(sender_id, receiver_id, amount))  

            # Commiting changes
            connection.commit()
            print(f"Sender balance after transaction:",sender_balance)
            
            print(f"Receiver balance after transaction:",receiver_balance)
        
        else:
            print("Insufficient funds.")
       
    except Exception as e:
        connection.rollback()
        print(e)

In [8]:
transaction(2,1,54400)

Sender balance: 987892
Amount to be transfered: 54400
Receiver balance before transaction: 500
Sender balance after transaction: 933492
Receiver balance after transaction: 54900


In [9]:
transaction(1,3,40000)

Sender balance: 54900
Amount to be transfered: 40000
Receiver balance before transaction: 999
Sender balance after transaction: 14900
Receiver balance after transaction: 40999


### Closing connection

In [10]:
connection.close()

# 2. **Task: Order Processing**
You are developing an online shopping platform.
Create a SQL transaction that updates the inventory of products when a customer places an order.
Deduct the ordered quantity from the product stock and update the order status.
If any part of the transaction fails (e.g., insufficient stock), roll back the changes to ensure consistent data.

### Establishing connection to MySQL

In [11]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='online_shop')

### Creating a cursor object

In [12]:
cursor = connection.cursor()

### Creating tables

In [13]:
customers_table = ("""
CREATE TABLE customers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    surname VARCHAR(50)
    );
""")

products_table = ("""
CREATE TABLE products (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    price FLOAT,
    amount INT
    );
""")

orders_table = ("""
CREATE TABLE orders (
    id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    amount INT,
    status VARCHAR(50)
    );
""")

cursor.execute(customers_table)
cursor.execute(products_table)
cursor.execute(orders_table)

### Inserting values into tables

In [14]:
cursor.execute("""
INSERT INTO customers (
    name, surname
    ) VALUES                  
        ("Johny", "Bravo"),
        ("Michael", "Keaton"),
        ("Philip", "Smith"),
        ("Jack", "Bing"),
        ("Velma", "Vellington");
""")
cursor.execute("""
INSERT INTO products (
    name, price, amount
    ) VALUES                  
        ("Phone", 489.9, 15),
        ("PC", 780, 10),
        ("Desk", 220.5, 100),
        ("Chair", 150, 150),
        ("Tablet", 359.2, 45);
""")

In [15]:
connection.commit()

### Transaction:

In [16]:
def order (customer_id:int,product_id:int,amount:int):
    try:
        cursor.execute("SELECT amount FROM products WHERE id = %s;",(product_id,))
        product_amount = cursor.fetchone()[0]
        
        print(f"Product amount:",product_amount)
        print(f"Amount to be sold:",amount)
        
        # Updating orders table
        cursor.execute("INSERT INTO orders (customer_id, product_id, amount, status) VALUES (%s,%s,%s,'pending')",(customer_id, product_id, amount))
        
        if amount <= product_amount:
            
            # Updating products stock
            product_amount -= amount
            cursor.execute("UPDATE products SET amount = %s WHERE id = %s;",(product_amount, product_id,))
            
            # Updating orders table
            cursor.execute("UPDATE orders SET status = 'complete' WHERE id = LAST_INSERT_ID()")  

            # Commiting changes
            connection.commit()
            print(f"Products amount after order:",product_amount)
        
        else:
            print("Insufficient product stock.")
            #Updating orders table
            cursor.execute("UPDATE orders SET status = 'rejected' WHERE id = LAST_INSERT_ID()")
            # Commiting changes
            connection.commit()
       
    except Exception as e:
        connection.rollback()
        print(e)

In [17]:
order(1,1,20)

Product amount: 15
Amount to be sold: 20
Insufficient product stock.


In [18]:
order(3,1,12)

Product amount: 15
Amount to be sold: 12
Products amount after order: 3


In [19]:
order(2,3,2)

Product amount: 100
Amount to be sold: 2
Products amount after order: 98


### Closing connection

In [20]:
connection.close()

# 3. **Task: Employee Salary Adjustment**
Your HR department needs to apply a salary adjustment to all employees based on their performance.
Design a SQL transaction that updates the salary of each employee with the corresponding adjustment and logs the changes. If there is any issue while updating salaries, roll back the changes to maintain the previous salary data.

### Establishing connection to MySQL

In [21]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='employee_salaries')

### Creating a cursor object

In [22]:
cursor = connection.cursor()

### Creating tables

In [23]:
employees_table = ("""
CREATE TABLE employees (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    surname VARCHAR(50),
    salary INT
    );
""")

salary_adjustment_table = ("""
CREATE TABLE salary_adjustments (
    id INT AUTO_INCREMENT PRIMARY KEY,
    employee_id INT,
    salary_adjustment INT,
    timestamp TIMESTAMP
    );
""")

cursor.execute(employees_table)
cursor.execute(salary_adjustment_table)

### Inserting values into tables

In [24]:
cursor.execute("""
INSERT INTO employees (
    name, surname, salary
    ) VALUES                  
        ("Johny", "Bravo", 1000),
        ("Michael", "Keaton", 850),
        ("Philip", "Smith", 1250),
        ("Jack", "Bing", 2200),
        ("Velma", "Vellington", 600);
""")

In [25]:
connection.commit()

### Transaction:

In [26]:
def salary (employee_id:int,salary_adjustment:int):
    try:
        cursor.execute("SELECT salary FROM employees WHERE id = %s;",(employee_id,))
        employee_salary = cursor.fetchone()[0]
        
        print(f"Employee:",employee_id)
        print(f"Salary before adjustment:",employee_salary)
        
        # Updating salary adjustments table
        cursor.execute("INSERT INTO salary_adjustments (employee_id, salary_adjustment) VALUES (%s,%s)",(employee_id, salary_adjustment))  
        # Commiting changes
        connection.commit()

        # Updating employees table
        employee_salary += salary_adjustment
        cursor.execute("UPDATE employees SET salary = %s WHERE id = %s",(employee_salary,employee_id))  

        # Commiting changes
        connection.commit()
        print(f"New salary:",employee_salary)
       
    except Exception as e:
        connection.rollback()
        print(e)

In [27]:
salary(3,-50)

Employee: 3
Salary before adjustment: 1250
New salary: 1200


In [28]:
salary(3,200)

Employee: 3
Salary before adjustment: 1200
New salary: 1400


### Closing connection

In [29]:
connection.close()

# 4. **Task: Flight Reservation**
You are building a flight reservation system. Create a SQL transaction that reserves a seat for a passenger on a flight.
Deduct the seat availability, update the reservation record, and log the booking details.
If any part of the transaction fails (e.g., seat already taken), roll back the changes and notify the user of the failure.

### Establishing connection to MySQL

In [30]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='seats_reservations')

### Creating a cursor object

In [31]:
cursor = connection.cursor()

### Creating tables

In [32]:
passengers_table = ("""
CREATE TABLE passengers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    surname VARCHAR(50)
    );
""")

seats_table = ("""
CREATE TABLE seats (
    seat_number INT AUTO_INCREMENT PRIMARY KEY,
    availability VARCHAR(50)
    );
""")

reservations_table = ("""
CREATE TABLE reservations (
    id INT AUTO_INCREMENT PRIMARY KEY,
    passenger_id INT,
    seat_number INT,
    status VARCHAR(50)
    );
""")

cursor.execute(passengers_table)
cursor.execute(seats_table)
cursor.execute(reservations_table)

### Inserting values into tables

In [33]:
cursor.execute("""
INSERT INTO passengers (
    name, surname
    ) VALUES                  
        ("Johny", "Bravo"),
        ("Michael", "Keaton"),
        ("Philip", "Smith"),
        ("Jack", "Bing"),
        ("Velma", "Vellington");
""")
for i in range(50):
    cursor.execute("""
    INSERT INTO seats (
        seat_number, availability
        ) VALUES                  
            (%s, 'open');
    """,(i+1,))
    i += 1

In [34]:
connection.commit()

### Transaction:

In [35]:
def seat_reservation (passenger_id:int,seat_number:int):
    try:
        cursor.execute("SELECT seat_number,availability FROM seats WHERE seat_number = %s;",(seat_number,))
        seat_availability = cursor.fetchone()[1]
        
        print(f"Chosen seat:",seat_number)
        print(f"Seat availability:",seat_availability)
        
        # Updating reservations table
        cursor.execute("INSERT INTO reservations (passenger_id, seat_number, status) VALUES (%s,%s,'pending')",(passenger_id, seat_number))  
        # Commiting changes
        connection.commit()
            
        if seat_availability == 'open':
            
            # Updating seats table
            seat_availability = 'reserved'
            cursor.execute("UPDATE seats SET availability = %s WHERE seat_number = %s;",(seat_availability, seat_number,))
            
            # Updating reservations table
            cursor.execute("UPDATE reservations SET status = 'successful' WHERE id = LAST_INSERT_ID()")  

            # Commiting changes
            connection.commit()
            print(f"Reservation successful")
        
        else:
            print("Reservation declined. Seat is occupied.")
            #Updating reservations table
            cursor.execute("UPDATE reservations SET status = 'declined' WHERE id = LAST_INSERT_ID()")
            # Commiting changes
            connection.commit()
       
    except Exception as e:
        connection.rollback()
        print(e)

In [36]:
seat_reservation(2,2)

Chosen seat: 2
Seat availability: open
Reservation successful


In [37]:
seat_reservation(3,2)

Chosen seat: 2
Seat availability: reserved
Reservation declined. Seat is occupied.


### Closing connection

In [38]:
connection.close()

# 5. **Task: Product Returns**
You are developing an e-commerce platform. Implement a SQL transaction to process product returns initiated by customers.
Update the product's return status, refund the customer's payment, and log the return details.
If any part of the transaction fails (e.g., invalid return request), roll back the changes and handle the error gracefully.

### Establishing connection to already created database in MySQL

In [39]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='online_shop')

### Creating a cursor object

In [40]:
cursor = connection.cursor(buffered=True)

### Creating new table

In [41]:
returns_table = ("""
CREATE TABLE returns (
    id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    amount INT,
    refund FLOAT,
    status VARCHAR(50)
    );
""")

cursor.execute(returns_table)

In [42]:
connection.commit()

### Transaction:

In [43]:
def return_product (customer_id:int,product_id:int,amount:int):
    try:
        cursor.execute("SELECT id, amount, status FROM orders WHERE customer_id = %s AND product_id = %s AND status = 'complete';",(customer_id, product_id,))
        results = cursor.fetchone()
        
        order_id = results[0]
        products_sold = results[1]
        order_status = results[2]

        cursor.execute("SELECT price, amount FROM products WHERE id = %s;",(product_id,))
        results_products = cursor.fetchone()
        refund = results_products[0] * products_sold
        products_in_store = results_products[1]

        print(f"Customer:",customer_id)        
        print(f"Product identification of the product to be returned:",product_id,". Amount:",amount)
        print(f"Refund:",refund)
        
        # Updating returns table
        cursor.execute("INSERT INTO returns (customer_id, product_id, amount, refund, status) VALUES (%s,%s,%s,%s,'pending')",(customer_id, product_id, amount, refund))
        
        if order_status == 'complete':
            
            if amount <= products_sold:    
                
                # Updating products stock
                products_in_store += amount
                cursor.execute("UPDATE products SET amount = %s WHERE id = %s;",(products_in_store, product_id,))

                # Updating orders table
                cursor.execute("UPDATE returns SET status = 'refunded' WHERE id = LAST_INSERT_ID()")  

                # Commiting changes
                connection.commit()
                print("Product return completed. Products amount in store after return: ",products_in_store)  
                
            else:
                print("Return declined. Too many products asked to be returned.")
        
        else:
            print("Return declined. No such order was completed.")
            #Updating returns table
            cursor.execute("UPDATE returns SET status = 'rejected' WHERE id = LAST_INSERT_ID()")
            # Commiting changes
            connection.commit()
       
    except Exception as e:
        connection.rollback()
        print("Return declined. No such order was completed.")

In [44]:
return_product(1,1,5)

Return declined. No such order was completed.


In [45]:
return_product(2,3,1)

Customer: 2
Product identification of the product to be returned: 3 . Amount: 1
Refund: 441.0
Product return completed. Products amount in store after return:  99


### Closing connection

In [46]:
connection.close()

# 6. **Task: Student Enrollment**
Design a SQL transaction to handle the enrollment of new students in a university.
Update the student enrollment records, assign courses, and update the tuition fee payment status. 
If any part of the transaction fails (e.g., course limit reached), roll back the changes to ensure no invalid enrollments are recorded.

### Establishing connection to MySQL

In [47]:
connection = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='university')

### Creating a cursor object

In [48]:
cursor = connection.cursor()

### Creating tables

In [49]:
students_table = ("""
CREATE TABLE students (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    surname VARCHAR(50),
    courses_enrolled INT DEFAULT 0,
    tuition_fee INT DEFAULT 0
    );
""")

courses_table = ("""
CREATE TABLE courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    price INT
    );
""")

students_courses_table = ("""
CREATE TABLE students_courses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    student_id INT,
    course_id INT
    );
""")

cursor.execute(students_table)
cursor.execute(courses_table)
cursor.execute(students_courses_table)

### Inserting values into tables

In [50]:
cursor.execute("""
INSERT INTO courses (
    name, price
    ) VALUES                  
        ("English", 400),
        ("Math", 650),
        ("Physics", 780),
        ("Spanish", 200),
        ("Chemistry", 770),
        ("Computer Science", 510),
        ("Biology", 620);
""")

In [51]:
connection.commit()

### Transaction:

In [52]:
def update_student_courses (student_id:int,courses_list:list):
    try:
        cursor.execute("SELECT courses_enrolled FROM students WHERE id = %s;",(student_id,))
        courses_enrolled = cursor.fetchone()[0]
        courses_count = len(courses_list)
        
        if (courses_enrolled + courses_count) <= 4:
            tuition_fee = 0
            for course in courses_list:
                cursor.execute("INSERT INTO students_courses (student_id, course_id) VALUES (%s,%s)",(student_id, course,))
                cursor.execute("SELECT price FROM courses WHERE id = %s;",(course,))
                tuition_fee += cursor.fetchone()[0]

            # Updating students table
            cursor.execute("UPDATE students SET courses_enrolled = %s , tuition_fee = %s WHERE id = %s;",(courses_count, tuition_fee,student_id))

            # Commiting changes
            connection.commit()
            print("Student successfuly enrolled to courses.")

        else: 
            print("Enrollment failed: Too many courses were chosen. LIMIT is 4.")
   
    except Exception as e:
        connection.rollback()
        print(f"Course update failed:{e}")
        
def student_enrollment (name:str,surname:str,courses_list:list):
    try:
        courses_count = len(courses_list)
        if courses_count <= 4:
            # Updating students table
            cursor.execute("INSERT INTO students (name, surname) VALUES (%s,%s)",(name, surname))
            connection.commit()

            # Checking for student_id
            cursor.execute("SELECT LAST_INSERT_ID()")
            student_id = cursor.fetchone()[0]

            update_student_courses(student_id,courses_list)
        else: 
            print("Enrollment failed: Too many courses were chosen. LIMIT is 4.")
            
    except Exception as e:
        connection.rollback()
        print(f"Enrollment failed:{e}")

In [53]:
student_enrollment("Johny","Bravo",[1,2,3])

Student successfuly enrolled to courses.


In [54]:
student_enrollment("Linda","Bold",[5,2,4])

Student successfuly enrolled to courses.


In [55]:
update_student_courses(2,[3,6,1])

Enrollment failed: Too many courses were chosen. LIMIT is 4.


In [56]:
update_student_courses(2,[1])

Student successfuly enrolled to courses.


### Closing connection

In [57]:
connection.close()

# Non SQL database:

In Python, design your own design of the database (using the files) and perform some practical examples with it. 

In [58]:
import json

In [59]:
animal1 = {
    "id":1,
    "name": "Bingo",
    "species": "dog",
    "age": 8    
}
animal2 = {
    "id":2,
    "name": "Jango",
    "species": "dog",
    "age": 2    
}
animal3 = {
    "id":3,
    "name": "Grey",
    "species": "cat",
    "age": 3    
}

animals = list()

animals.append(animal1)
animals.append(animal2)
animals.append(animal3)

In [60]:
with open("shelter.json","w") as shelter_file:
    json.dump(animals,shelter_file)

In [61]:
with open("shelter.json","r") as shelter_file:
    animals_data = json.load(shelter_file)

In [62]:
animals_data

[{'id': 1, 'name': 'Bingo', 'species': 'dog', 'age': 8},
 {'id': 2, 'name': 'Jango', 'species': 'dog', 'age': 2},
 {'id': 3, 'name': 'Grey', 'species': 'cat', 'age': 3}]

In [63]:
class AnimalsDB:
    
    def __init__(self):
        self.load_animals_from_file()
    def get_last_id(self):
        last_id = 0
        for animal in self.animals_data:
            if last_id < animal["id"]:
                last_id = animal["id"]
        return last_id
                
    def insert(self,name:str,species:str,age:int):
        new_id = self.get_last_id() + 1
        animal = {"id":new_id,"name":name,"species":species,"age":age}
        self.animals_data.append(animal)
        
    def commit(self):
        with open("shelter.json","w") as json_f:
            json.dump(self.animals_data,shelter_file)
            
    def rollback(self):
        self.load_animals_from_file()

    def load_animals_from_file(self):
         with open("shelter.json","r") as shelter_file:
            self.animals_data = json.load(shelter_file)
            
    def update(self,id:int,name=None,species=None,age=None):
        animal_update = None
        for animal in self.animals_data:
            if animal["id"] == id:
                animal_update = animal
                break
        if animal_update == None:
            return False
        
        if not (name == None):
            animal["name"] = name
        if not (species == None):
            animal["species"] = species
        if not (age == None):    
            animal["age"] = age
        return True        

In [64]:
animals_db = AnimalsDB()

In [65]:
animals_db.insert("Honey","Horse",13)

In [66]:
animals_db.animals_data

[{'id': 1, 'name': 'Bingo', 'species': 'dog', 'age': 8},
 {'id': 2, 'name': 'Jango', 'species': 'dog', 'age': 2},
 {'id': 3, 'name': 'Grey', 'species': 'cat', 'age': 3},
 {'id': 4, 'name': 'Honey', 'species': 'Horse', 'age': 13}]

In [67]:
animals_db.rollback()

In [68]:
animals_db.animals_data

[{'id': 1, 'name': 'Bingo', 'species': 'dog', 'age': 8},
 {'id': 2, 'name': 'Jango', 'species': 'dog', 'age': 2},
 {'id': 3, 'name': 'Grey', 'species': 'cat', 'age': 3}]

In [69]:
animals_db.update(2,age=4)

True

In [70]:
animals_db.animals_data

[{'id': 1, 'name': 'Bingo', 'species': 'dog', 'age': 8},
 {'id': 2, 'name': 'Jango', 'species': 'dog', 'age': 4},
 {'id': 3, 'name': 'Grey', 'species': 'cat', 'age': 3}]